In [1]:
!pip3 install -q ktrain

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
train_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23train.csv") #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23test.csv") #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_2'],test_size=0.15,random_state=42) #training and validation split
target_names=[0,1,2] #names of classes

In [4]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [5]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 25
	95percentile : 47
	99percentile : 53


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 23
	95percentile : 45
	99percentile : 52


In [6]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 321 steps, validate for 11 steps
Epoch 1/4
321/321 [==============================] - 470s 1s/step - loss: 0.9177 - accuracy: 0.5955 - val_loss: 0.8193 - val_accuracy: 0.6794
Epoch 2/4
321/321 [==============================] - 452s 1s/step - loss: 0.7567 - accuracy: 0.6897 - val_loss: 0.7578 - val_accuracy: 0.6971
Epoch 3/4
321/321 [==============================] - 451s 1s/step - loss: 0.6771 - accuracy: 0.7199 - val_loss: 0.8020 - val_accuracy: 0.6853
Epoch 4/4
321/321 [==============================] - 451s 1s/step - loss: 0.4467 - accuracy: 0.8282 - val_loss: 0.8360 - val_accuracy: 0.6912


In [7]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.74      0.75      0.74       114
           1       0.79      0.73      0.76       172
           2       0.38      0.46      0.42        54

    accuracy                           0.69       340
   macro avg       0.63      0.65      0.64       340
weighted avg       0.71      0.69      0.70       340



array([[ 85,  14,  15],
       [ 21, 125,  26],
       [  9,  20,  25]])

In [8]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [9]:
test_b_text=list(test_b['text']) #get text for testing
test_b_label=list(test_b['task_2']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [10]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [11]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [12]:
from sklearn.metrics import classification_report

In [13]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.71      0.86      0.78        93
           1       0.66      0.79      0.72       124
           2       0.39      0.15      0.22        71

    accuracy                           0.66       288
   macro avg       0.59      0.60      0.57       288
weighted avg       0.61      0.66      0.62       288

